In [1]:
# drive mount. colab에 내 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

# clone git repo
!git clone https://github.com/hila-chefer/Transformer-Explainability.git

# change directory
import os
os.chdir(f'./Transformer-Explainability')

#!pip install torch==1.7.0 torchvision==0.8.1  # 일반 GPU/CPU를 사용하는 경우
!pip install torch==1.7.0+cu110 torchvision==0.8.1+cu110 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html  # GPU A100을 사용하는 경우
!pip install transformers==3.5.1
!pip install captum
!pip install matplotlib==3.2.2 &> /dev/null

Mounted at /content/drive
Cloning into 'Transformer-Explainability'...
remote: Enumerating objects: 377, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 377 (delta 127), reused 74 (delta 74), pack-reused 225
Receiving objects: 100% (377/377), 3.83 MiB | 18.78 MiB/s, done.
Resolving deltas: 100% (190/190), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████▌        | 834.1 MB 120.2 MB/s eta 0:00:03tcmalloc: large alloc 1147494400 bytes == 0x39fae000 @  0x7fd3975f0615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |█████████████████████████████▊  

In [2]:
##### 시각화 관련 필수 라이브러리
import torch

from transformers import BertTokenizer
from transformers import AutoTokenizer  # bert 모델에 따라 알맞은 tokenizer를 자동으로 로드

from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification

from captum.attr import visualization # XAI관련 라이브러리의 시각화 함수
#####


from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup

from sklearn.metrics import accuracy_score

import os
import json
import pickle
import numpy as np
import random
from collections import OrderedDict

In [3]:
# GPU 찾기. 없으면 CPU로 동작
if torch.cuda.is_available():  
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

# check torch is available
print(torch.__version__)
print(torch.tensor([1.0, 2.0]).cuda())

There are 1 GPU(s) available.
We will use the GPU: A100-SXM4-40GB
1.7.0+cu110
tensor([1., 2.], device='cuda:0')


In [5]:
# 데이터셋 로드
input_ids, attention_masks, ratings = np.load("/content/drive/MyDrive/CS470_team_2in1/dataset/amazon_book_only500000.npy", allow_pickle=True)
"""
# 임시로 1000개만 사용
input_ids = input_ids[:1000]
attention_masks = attention_masks[:1000]
ratings = ratings[:1000]
"""

# Convert to tensors and make dataset
input_ids = torch.cat(input_ids.tolist(), dim=0)
attention_masks = torch.cat(attention_masks.tolist(), dim=0)
labels = torch.tensor(ratings.tolist())
dataset = TensorDataset(input_ids, attention_masks, labels)

# split validation set
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32.
batch_size = 8

# Create DataLoader
train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size)
val_dataloader = DataLoader(val_dataset, sampler = SequentialSampler(val_dataset), batch_size = batch_size)

In [6]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
model.cuda()

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (add1): Add()
      (add2): Add()
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (matmul1): MatMul()
              (matmul2): MatMul()
              (softmax): Softmax(dim=-1)
              (add): Add()
              (

In [7]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)

epochs = 10 # The BERT authors recommend between 2 and 4. 
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

criterion = torch.nn.CrossEntropyLoss(reduction='none')

In [ ]:
results = {
        'train_loss': [],
        'train_f1': [],
        'train_acc': [],
        'val_loss': [],
        'val_f1': [],
        'val_acc': [],
    }

best_val_acc = 0
best_model_state_dict = None

for epoch in range(0, epochs):

    print('\n======== Epoch {:} / {:} ========\n'.format(epoch + 1, epochs))

    # train
    e_train_loss = 0
    e_train_acc = 0
    model.train()
    for batch in train_dataloader:
        b_input_ids, b_input_mask, b_labels = batch
        b_input_ids, b_input_mask, b_labels = b_input_ids.to(device), b_input_mask.to(device), b_labels.to(device)

        preds = model(input_ids=b_input_ids, attention_mask=b_input_mask)[0]
        e_train_acc += accuracy_score(preds.argmax(dim=1).cpu(), b_labels.cpu(), normalize=False)

        loss = criterion(preds, b_labels).sum()
        e_train_loss += loss.item()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Clip the norm of the gradients to 1.0. This is to help prevent the "exploding gradients" problem.

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        scheduler.step()
    e_train_loss /= train_size
    e_train_acc /= train_size
    results['train_loss'].append(e_train_loss)
    results['train_acc'].append(e_train_acc)


    # validation
    model.eval()
    #with torch.no_grad(): # 왜 에러?
    e_val_loss = 0
    e_val_acc = 0
    for batch in val_dataloader:
      b_input_ids, b_input_mask, b_labels = batch
      b_input_ids, b_input_mask, b_labels = b_input_ids.to(device), b_input_mask.to(device), b_labels.to(device)

      preds = model(input_ids=b_input_ids, attention_mask=b_input_mask)[0]
      e_val_acc += accuracy_score(preds.argmax(dim=1).cpu(), b_labels.cpu(), normalize=False)

      loss = criterion(preds, b_labels).sum()
      e_val_loss += loss.item()
    e_val_loss /= val_size
    e_val_acc /= val_size
    results["val_acc"].append(e_val_acc)
    results["val_loss"] = e_val_loss


    # save best model weights
    if best_val_acc < e_val_acc:
      best_val_acc = e_val_acc
      best_model_state_dict = OrderedDict({k: v.cpu() for k, v in model.state_dict().items()})
    
    
    print('train loss:',e_train_loss)
    print('train acc:',e_train_acc)
    print('val loss:',e_val_loss)
    print('val acc:',e_val_acc)


======== Epoch 1 / 10 ========



In [ ]:
model.load_state_dict(best_model_state_dict)
torch.save(best_model_state_dict, "/content/drive/MyDrive/CS470_team_2in1/colab/model/amazon_book_only500000_ver1.pt")

In [ ]:
# test generating explanation
def interpret_sentence(model, expl_generator, tokenizer, sentence, target_class=None, padding=False):
  # tokenize 해서 token id와 attention mask를 얻기
  if padding : encoding = tokenizer(sentence, max_length=512, padding="max_length", truncation=True, return_tensors='pt')
  else : encoding = tokenizer(sentence, return_tensors='pt')
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())

  # 모델 출력
  output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
  pred_class = output.argmax(dim=-1).item()
  output = output.detach().cpu().numpy()
  if target_class==None: target_class = pred_class  # 일단은 예측 라벨에 대해서 설명 생성

  # 설명 생성
  expl = expl_generator.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=11, index=target_class)[0]
  expl = expl.detach().cpu().numpy()
  expl = (expl - expl.min()) / (expl.max() - expl.min()) # normalize scores
  # normalize 방식이 https://captum.ai/tutorials/IMDB_TorchText_Interpret 랑 다른데? 링크는 l2, 코드는 최댓값을 1로.

  return tokens, output, expl, pred_class, target_class

epxl_generator = Generator(model)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
interpret_sentence(model, epxl_generator, tokenizer, ["good product for the price. used very day and no problem"], target_class=None)